In [ ]:
using MKL
using PhysicalUnits, LinearAlgebra
using MoireIVC.LLHF
using MoireIVC.LLHF_Plot
using MoireIVC.LLTDHF
using CairoMakie
CairoMakie.activate!()

In [ ]:
N1 = 18; N2 = 18
num_para = LLHF_init_with_lambda(1.0; N1 = N1, N2 = N2, LL = 0);
LLHF_change_lambda!(num_para, 0.0);
LLHF.H0_C3_T!(num_para, 0.1);
LLHF_plot_band_3D(LLHF.VP_solution(num_para,1); para=num_para)

In [ ]:
symmetry = [LLHF.Rot3(0); LLHF.PT(0,:T)]
@time ρ = LLHF_solve(num_para; coherence = 0.0, 
    method = :hf_SC_mixing!, mixing_rate = :dynamic,
    error_tolerance = 1E-10, iteration_steps = 100, 
    procession_steps = 100, step_processions = symmetry,
    stepwise_output = true, process_output = true
);

In [ ]:
LLHF_plot_band_3D(ρ; para=num_para)

In [ ]:
LLHF_plot_band(ρ; para=num_para)

In [ ]:
LLHF_plot_phase(ρ; para=num_para)

In [ ]:
LLHF_plot_Sz(ρ; para=num_para)

In [ ]:
LLHF_plot_Berrycurvature(ρ; para=num_para)

In [ ]:
# arrowcolor can be :Sz or :n
LLHF_plot_realspace(ρ; para=num_para, arrowcolor = :n,
    N = 18, xlimits = (-1.2, 1.2), ylimits = (-1.2, 1.2), 
    arrowscale = 1, arrowsize = 4
)

In [ ]:
GS = LLTDHF.TDHF_groundstateanalysis(ρ, num_para);
q1range = 0:5
omegas = zeros(ComplexF64, 4, length(q1range));
for i in eachindex(q1range)
    println(i)
    q1 = q1range[i]; q2=0
    @time Vspin = TDHF_V_matrix_spin(q1, q2, num_para);
    M = TDHF_ZEXV(q1, q2, GS; Vspin = Vspin);
    vals = TDHF_solve(M)[1:8]
    perm = sortperm(vals; by = real)
    omegas[:,i] = vals[perm[5:8]]
end

In [ ]:
magnon_spectrum = Figure()
ax = Axis(magnon_spectrum[1,1];
    xlabel = "q1", ylabel = "ω(W0)"
)
for i = 1:4
    scatterlines!(ax, q1range, real.(omegas[i,:])./num_para.system.W0)
end
magnon_spectrum